In [2]:
%load_ext autoreload
%autoreload 2


import notebook_path
import pandas as pd
from config import *
from pathlib import Path
import pickle
from plotting import view_spectrogram
from util import Dataset


In [3]:
annotations_df = pd.read_csv(ANNOTATIONS + "\\initial_dataset_7depl.csv")
annotations_df

,peak_time,xcorrcoef,min_t,max_t,min_f,max_f,recording,deployment,site,label,datetime
0,364.624000,0.584717,364.160667,365.087333,5531.00000,10125.000000,1_20230317_070000.wav,1.0,1.0,fast_trill_6khz,2023-03-17 07:00:00
1,215.178667,0.658409,214.715333,215.642000,5531.00000,10125.000000,1_20230319_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-19 06:30:00
2,526.810667,0.604653,526.347333,527.274000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
3,544.922667,0.663969,544.459333,545.386000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
4,449.312000,0.610269,448.848667,449.775333,5531.00000,10125.000000,1_20230320_073000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 07:30:00
...,...,...,...,...,...,...,...,...,...,...,...
2185,384.869333,0.654130,384.514667,385.224000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2186,386.693333,0.650236,386.338667,387.048000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2187,445.056000,0.651619,444.701333,445.410667,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2188,446.906667,0.683734,446.552000,447.261333,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00


In [4]:
ds = Dataset()

recs = ds.get_recordings(1, 1)

grouped_df = annotations_df.groupby("recording")


recs_and_annotations = []

annotated = 0
for rec in recs:
    
    if rec in grouped_df.groups:
        print(len(grouped_df.get_group(rec)))
        annotated += 1
        
        recs_and_annotations.append( (rec, grouped_df.get_group(rec)) )

27
54
53
4
6
7
2
28
4
7
4
9
2
1
31
4
1
2
17
3
3
2
3
10
33
3
2
13
3
2
1
23
2
1
6
5
52
13
5
1
1
54
6
9
2
45
1
1
45
26


In [5]:
label, df = recs_and_annotations[0]
print(df)

       peak_time  xcorrcoef       min_t       max_t        min_f        max_f  \
949   273.584000   0.715624  273.261333  273.906667  1202.470337  4831.221191   
950   281.418667   0.711718  281.096000  281.741333  1202.470337  4831.221191   
951   292.693333   0.763661  292.370667  293.016000  1202.470337  4831.221191   
952   299.984000   0.724239  299.661333  300.306667  1202.470337  4831.221191   
953   313.936000   0.724176  313.613333  314.258667  1202.470337  4831.221191   
954   326.869333   0.776954  326.546667  327.192000  1202.470337  4831.221191   
955   350.277333   0.715920  349.954667  350.600000  1202.470337  4831.221191   
956   358.149333   0.685052  357.826667  358.472000  1202.470337  4831.221191   
957   363.888000   0.702963  363.565333  364.210667  1202.470337  4831.221191   
958   369.850667   0.707953  369.528000  370.173333  1202.470337  4831.221191   
959   375.706667   0.750953  375.384000  376.029333  1202.470337  4831.221191   
960   383.813333   1.000000 

In [29]:
import librosa 
from plotting import view_spectrogram
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from tqdm import tqdm


sr = 22_000
chunk_len_s = 10 # seconds

chunk_len = int(librosa.time_to_frames(chunk_len_s, sr=sr))
    

for (rec, df) in annotations_df[:100].groupby('recording'): 
    direct = f"../untracked/labels/{rec}"
    Path(direct).mkdir(exist_ok=True)    
    
    p = Path(ds.get_data_path(1, 1)) / rec
    y, sr = librosa.load(p, sr=sr)
    S = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    
    
    for start_idx in range(0, S_db.shape[1], chunk_len):
        
        start_time = librosa.frames_to_time(start_idx, sr=sr)
        end_time = librosa.frames_to_time(start_idx + chunk_len, sr=sr)
        
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.set(title=f"{rec} | {start_time} -> {end_time}")
                
        # cut out chunk
        S_chunk = S_db[:, start_idx : start_idx+chunk_len]
    
        # show spect
        img = librosa.display.specshow(S_chunk, y_axis="linear", ax=ax, sr=sr)

        # draw annotations if appopriate        
        for index, row in df.iterrows():
            if row['min_t'] > start_time and row['min_t'] < end_time:
                x =  int(librosa.time_to_frames(row['min_t'] - start_time, sr=sr))
                width = int(librosa.time_to_frames(row['max_t'] - start_time, sr=sr)) - x
                
                y =  row['min_f']
                height =  row['max_f'] - y
                
                rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='red', facecolor='none')
                ax.add_patch(rect)
                
                ax.text(x, y, row["label"], color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

        plt.savefig(direct + f"/{rec}_{start_time}-{end_time}.png")
        plt.close()
        
        
    

C:\Users\taznk\AppData\Local\Temp\ipykernel_14404\21514215.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(p, sr=sr)
c:\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\acoustic-AL-dataset\\site01\\deployment_001\\Data\\1_20230517_073000.wav'

In [1]:
import pandas as pd

df = pd.read_csv("../output/annotations/initial_dataset_7depl.csv")

,peak_time,xcorrcoef,min_t,max_t,min_f,max_f,recording,deployment,site,label,datetime
0,364.624000,0.584717,364.160667,365.087333,5531.00000,10125.000000,1_20230317_070000.wav,1.0,1.0,fast_trill_6khz,2023-03-17 07:00:00
1,215.178667,0.658409,214.715333,215.642000,5531.00000,10125.000000,1_20230319_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-19 06:30:00
2,526.810667,0.604653,526.347333,527.274000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
3,544.922667,0.663969,544.459333,545.386000,5531.00000,10125.000000,1_20230320_063000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 06:30:00
4,449.312000,0.610269,448.848667,449.775333,5531.00000,10125.000000,1_20230320_073000.wav,1.0,1.0,fast_trill_6khz,2023-03-20 07:30:00
...,...,...,...,...,...,...,...,...,...,...,...
2185,384.869333,0.654130,384.514667,385.224000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2186,386.693333,0.650236,386.338667,387.048000,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2187,445.056000,0.651619,444.701333,445.410667,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00
2188,446.906667,0.683734,446.552000,447.261333,205.69162,617.953796,1_20230908_070000.wav,7.0,1.0,upsweep_500hz,2023-09-08 07:00:00


In [13]:
import numpy as np
y = np.zeros(shape=(10, 4), dtype=np.int32)
y[5]

array([0, 0, 0, 0])